# Try PoPS in Jupyter Notebook

### Intro to Jupyter Notebook

This is a quick introduction into Jupyter Notebook.  Use Shift+Enter to execute cells.

In [ ]:
a = 6
b = 7
c = a * b
print("Answer is", c)

Python code can be mixed with command line code (Bash).
It is enough just to prefix the command line with an exclamation mark:

In [ ]:
!echo "Answer is $c"

### Setting up GRASS GIS session


In [ ]:
from IPython.display import Image
from helper import initialize_GRASS_notebook

initialize_GRASS_notebook(binary="grass",
                          grassdata="data/grassdata", location="PoPS_SOD_tutorial",
                          mapset="tutorial")


import grass.script as gs
from helper import show_interactively, show